In [47]:
import dspy
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import List
from datetime import datetime
from pymongo import MongoClient

load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [89]:
# MongoDB connection
client = MongoClient(os.getenv('MONGODB_URI'))
db = client['renai']
collection = db['todos']

In [49]:
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [50]:
lm = dspy.LM('gemini/gemini-2.0-flash-exp', api_key=GEMINI_API_KEY)
dspy.configure(lm=lm)

In [101]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        # Fetch records from MongoDB
        records = collection.find()

        corpus = []
        for record in records:
            context = ""
            for key, value in record.items():
                if key == "_id":
                    continue
                # Convert key to proper case
                proper_case_key = key.replace('_', ' ').title().replace(' ', '')
                context += f"{proper_case_key}: {value}\n"
            corpus.append(context)
            
        print(corpus)
        embedder = dspy.Embedder('gemini/text-embedding-004')
        search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=5)
        context = search(question).passages
        return self.respond(context=context, question=question)


In [102]:
rag = RAG()
rag(question="Who do i need to re send engagement survey to?")

['Description: Update onboarding process to include shared document\nAssignee: Speaker 1\nDeadline: None\nStatus: pending\nMentionedBy: Speaker 1\n', 'Description: Follow up on interview spreadsheet customer contacts\nAssignee: Unassigned\nDeadline: None\nStatus: pending\nMentionedBy: Speaker 2\n', 'Description: Run engagement survey monthly in Q3\nAssignee: Speaker 1\nDeadline: None\nStatus: pending\nMentionedBy: Speaker 1\n', 'Description: Resend engagement survey to Speaker 3\nAssignee: Jessica\nDeadline: None\nStatus: pending\nMentionedBy: Speaker 1\n', 'Description: Resend engagement survey to Speaker 4\nAssignee: Jessica\nDeadline: None\nStatus: pending\nMentionedBy: Speaker 1\n']


Prediction(
    reasoning='The question asks who needs to be resent the engagement survey. I need to look for tasks that have "engagement survey" in the description and identify the associated speakers. Tasks [1] and [2] mention resending the engagement survey. Task [1] is for Speaker 3 and Task [2] is for Speaker 4.',
    response='You need to resend the engagement survey to Speaker 3 and Speaker 4.'
)